In [1]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
 
# import the necessary packages
from minivggnet import MiniVGGNet
from sklearn.metrics import classification_report
from keras.optimizers import SGD
from keras.datasets import fashion_mnist
from keras.utils import np_utils
from keras import backend as K
from imutils import build_montages
import matplotlib.pyplot as plt
import numpy as np
import cv2
 
# initialize the number of epochs to train for, base learning rate,
# and batch size
NUM_EPOCHS = 25
INIT_LR = 1e-2
BS = 32

Using TensorFlow backend.


In [2]:
# grab the Fashion MNIST dataset (if this is your first time running
# this the dataset will be automatically downloaded)
print("[INFO] loading Fashion MNIST...")
((trainX, trainY), (testX, testY)) = fashion_mnist.load_data()

[INFO] loading Fashion MNIST...


In [3]:
# initialize the label names
labelNames = ["top", "trouser", "pullover", "dress", "coat",
	"sandal", "shirt", "sneaker", "bag", "ankle boot"]

In [4]:
from PIL import Image

In [6]:
trainX[0].shape

(28, 28)

In [5]:
img = Image.fromarray( trainX[0] , 'L')
img.show()

In [7]:
img_num = 10
for img_ind in range(img_num):
    img = trainX[img_ind]
    cv2.imshow(labelNames[trainY[img_ind]], img)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [8]:
img_num = 10
for img_ind in range(img_num):
    img = testX[img_ind]
    cv2.imshow(labelNames[testY[img_ind]], img)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [9]:
kernel = np.array([[-1, -1, -1],
                       [-1, 8, -1],
                       [-1, -1, -1]])

In [10]:
img = cv2.imread('signature.png')
cv2.imshow("original.png", img)
cv2.waitKey()
cv2.destroyAllWindows()

In [11]:
dst = cv2.filter2D(img,-1,kernel)
cv2.imshow("filtered.png", dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [12]:
trainX.flags.writeable = True
img_num = trainX.shape[0]
for img_ind in range(img_num):
    img = trainX[img_ind]
    dst = cv2.filter2D(img,-1,kernel)
    trainX[img_ind] = dst
trainX.flags.writeable = False

In [13]:
testX.flags.writeable = True
img_num = testX.shape[0]
for img_ind in range(img_num):
    img = testX[img_ind]
    dst = cv2.filter2D(img,-1,kernel)
    testX[img_ind] = dst
testX.flags.writeable = False

In [14]:
img_num = 10
for img_ind in range(img_num):
    img = trainX[img_ind]
    cv2.imshow(labelNames[trainY[img_ind]], img)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [15]:
img_num = 10
for img_ind in range(img_num):
    img = testX[img_ind]
    cv2.imshow(labelNames[testY[img_ind]], img)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [16]:
cv2.destroyAllWindows()

In [17]:
# if we are using "channels first" ordering, then reshape the design
# matrix such that the matrix is:
# 	num_samples x depth x rows x columns
if K.image_data_format() == "channels_first":
	trainX = trainX.reshape((trainX.shape[0], 1, 28, 28))
	testX = testX.reshape((testX.shape[0], 1, 28, 28))
 
# otherwise, we are using "channels last" ordering, so the design
# matrix shape should be: num_samples x rows x columns x depth
else:
	trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
	testX = testX.reshape((testX.shape[0], 28, 28, 1))

In [18]:
# scale data to the range of [0, 1]
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0
 
# one-hot encode the training and testing labels
trainY = np_utils.to_categorical(trainY, 10)
testY = np_utils.to_categorical(testY, 10)

In [19]:
# initialize the optimizer and model
print("[INFO] compiling model...")
opt = SGD(lr=INIT_LR, momentum=0.9, decay=INIT_LR / NUM_EPOCHS)
model = MiniVGGNet.build(width=28, height=28, depth=1, classes=10)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
 
# train the network
print("[INFO] training model...")
H = model.fit(trainX, trainY,
	validation_data=(testX, testY),
	batch_size=BS, epochs=NUM_EPOCHS)

[INFO] compiling model...
[INFO] training model...
Train on 60000 samples, validate on 10000 samples
Epoch 1/25
60000/60000 [==============================] - 102s 2ms/step - loss: 0.5387 - acc: 0.8212 - val_loss: 0.3117 - val_acc: 0.8864
Epoch 2/25
60000/60000 [==============================] - 98s 2ms/step - loss: 0.3479 - acc: 0.8769 - val_loss: 0.2738 - val_acc: 0.9000
Epoch 3/25
60000/60000 [==============================] - 100s 2ms/step - loss: 0.3019 - acc: 0.8926 - val_loss: 0.2677 - val_acc: 0.9016
Epoch 4/25
60000/60000 [==============================] - 101s 2ms/step - loss: 0.2774 - acc: 0.9003 - val_loss: 0.2476 - val_acc: 0.9112
Epoch 5/25
60000/60000 [==============================] - 102s 2ms/step - loss: 0.2616 - acc: 0.9077 - val_loss: 0.2417 - val_acc: 0.9116
Epoch 6/25
60000/60000 [==============================] - 102s 2ms/step - loss: 0.2524 - acc: 0.9105 - val_loss: 0.2366 - val_acc: 0.9127
Epoch 7/25
60000/60000 [==============================] - 101s 2ms/step 

In [20]:
# make predictions on the test set
preds = model.predict(testX)
 
# show a nicely formatted classification report
print("[INFO] evaluating network...")
print(classification_report(testY.argmax(axis=1), preds.argmax(axis=1),
	target_names=labelNames))
 
# plot the training loss and accuracy
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")

[INFO] evaluating network...
             precision    recall  f1-score   support

        top       0.86      0.89      0.87      1000
    trouser       1.00      0.98      0.99      1000
   pullover       0.86      0.92      0.89      1000
      dress       0.92      0.92      0.92      1000
       coat       0.86      0.87      0.87      1000
     sandal       0.98      0.98      0.98      1000
      shirt       0.80      0.73      0.77      1000
    sneaker       0.95      0.98      0.96      1000
        bag       0.99      0.98      0.99      1000
 ankle boot       0.98      0.96      0.97      1000

avg / total       0.92      0.92      0.92     10000



In [21]:
# initialize our list of output images
images = []
 
# randomly select a few testing fashion items
for i in np.random.choice(np.arange(0, len(testY)), size=(16,)):
	# classify the clothing
	probs = model.predict(testX[np.newaxis, i])
	prediction = probs.argmax(axis=1)
	label = labelNames[prediction[0]]
 
	# extract the image from the testData if using "channels_first"
	# ordering
	if K.image_data_format() == "channels_first":
		image = (testX[i][0] * 255).astype("uint8")
 
	# otherwise we are using "channels_last" ordering
	else:
		image = (testX[i] * 255).astype("uint8")
        
    	# initialize the text label color as green (correct)
	color = (0, 255, 0)
 
	# otherwise, the class label prediction is incorrect
	if prediction[0] != np.argmax(testY[i]):
		color = (0, 0, 255)
 
	# merge the channels into one image and resize the image from
	# 28x28 to 96x96 so we can better see it and then draw the
	# predicted label on the image
	image = cv2.merge([image] * 3)
	image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_LINEAR)
	cv2.putText(image, label, (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.75,
		color, 2)
 
	# add the image to our list of output images
	images.append(image)
 
# construct the montage for the images
montage = build_montages(images, (96, 96), (4, 4))[0]
 
# show the output montage
cv2.imshow("Fashion MNIST", montage)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [22]:
print("done")

done


In [23]:
scores = model.evaluate(testX, testY, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 92.07%


In [24]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [25]:
model.save('model.hdf5')

In [26]:
from keras.models import model_from_json

In [27]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [29]:
loaded_model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

In [30]:
score = loaded_model.evaluate(testX, testY, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

acc: 92.07%


In [31]:
from keras.models import load_model

In [32]:
full_model = load_model('model.hdf5')

In [33]:
score = full_model.evaluate(testX, testY, verbose=0)
print("%s: %.2f%%" % (full_model.metrics_names[1], score[1]*100))

acc: 92.07%
